# Adventure Works 2025 Sales Analysis
In this analysis we examine the Adventure Works database to answer business intelligence questions like:
Product categories and their metrics.
Trends in sales over time.
Regional depth metrics.

We will use SQL for the database connection and Python for the visualizations.

In [ ]:
import pandas as pd 
import matplotlib as plt
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

user = "SA"
password = quote_plus("AStIyChGLQQn0K")
server = 'localhost:1433'
database = 'AdventureWorks2025'
driver = quote_plus("ODBC Driver 18 for SQL server")


connection_string = (
    f"mssql+pyodbc://{user}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)


try:
    with engine.connect():
        print("Connection to SQL Server was successful")
except Exception as e:
    print("Could not connect", e)

Anslutning till SQL Server lyckades


C:\Users\pappa\AppData\Local\Temp\ipykernel_2580\3602509395.py:22: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  with engine.connect():
